<a href="https://colab.research.google.com/github/soheilpaper/-tft-2.4-ili9341-STM32/blob/master/youtube_subtitle/ASR_%26_Speaker_Diarization_with_OpenAI_Whisper_%26_Nvidia_Nemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ASR (OpenAI Whisper)

In this section, we'll use OpenAI newly release Whisper to convert a sample youtube podcast to words <> timestamps mapping.

In [2]:
!apt install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


In [3]:
!pip3 install torch torchvision torchaudio yt-dlp

In [4]:

#@title Add here the YT URL

link = 'https://m.youtube.com/watch?v=LI-_HpIDyoI' #@param {type:"string"}

#file_path = f'{folder_path}/WL_video.mp4' #@param {type:"string"}

# A 20mins podcast from YC official youtube channel.
!rm ./audio.wav
!yt-dlp -xv --audio-format wav  -o audio.wav -- {link} #https://www.youtube.com/watch?v=bp_kMA-eTsE

[debug] Command-line config: ['-xv', '--audio-format', 'wav', '-o', 'audio.wav', '--', 'https://m.youtube.com/watch?v=LI-_HpIDyoI']
[debug] Encodings: locale UTF-8, fs utf-8, pref UTF-8, out utf-8, error utf-8, screen utf-8
[debug] yt-dlp version stable@2023.12.30 from yt-dlp/yt-dlp [f10589e34] (pip)
[debug] Python 3.10.12 (CPython x86_64 64bit) - Linux-6.1.58+-x86_64-with-glibc2.35 (OpenSSL 3.0.2 15 Mar 2022, glibc 2.35)
[debug] exe versions: ffmpeg 4.4.2 (setts), ffprobe 4.4.2
[debug] Optional libraries: Cryptodome-3.20.0, brotli-1.1.0, certifi-2023.11.17, mutagen-1.47.0, requests-2.31.0, secretstorage-3.3.1, sqlite3-3.37.2, urllib3-2.0.7, websockets-12.0
[debug] Proxy map: {'colab_language_server': '/usr/colab/bin/language_service'}
[debug] Request Handlers: urllib, requests, websockets
[debug] Loaded 1798 extractors
[youtube] Extracting URL: https://m.youtube.com/watch?v=LI-_HpIDyoI
[youtube] LI-_HpIDyoI: Downloading webpage
[youtube] LI-_HpIDyoI: Downloading ios player API JSON
[y

In [5]:
!rm ./audio_16k.wav
!ffmpeg -i audio.wav -ac 1 -ar 16000 audio_16k.wav # Converting audio.wav to mono channel & 16K audio_16k.wav

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [6]:
# Installing Whisper and WhisperX

!pip install git+https://github.com/openai/whisper.git
!pip install git+https://github.com/m-bain/whisperX.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-aus92_9c
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-aus92_9c
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/m-bain/whisperX.git to /tmp/pip-req-build-720_7qv2
  Running command git clone --filter=blob:none --quiet https://github.com/m-bain/whisperX.git /tmp/pip-req-build-720_7qv2
  Resolved https://github.com/m-bain/whisperX.git to commit 8227807fa9e076901ea4b4fbbf79c9777a6f5e03
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/SYSTRAN/faster-whisper.git (to revision 0.10.0) to /tmp/pip-install-nqx5c32o/faster-whisper_23e2a279118a459a881e0632ed7ddb87
  Running command git clone --filter=blob:none --quiet https://gith

In [7]:
from whisper import load_model

# Large models result in considerably better and more aligned (words, timestamps) mapping.
model = load_model("large-v2")

# Beam size if None by default (Greedy Decoding). You can also set the
# beam_size to some number like 5. This will increase in better transcription
# quality but it'll increase runtime considerabley.
results = model.transcribe('./audio_16k.wav', beam_size=None)

In [8]:
# WhisperX results in better word timestamps by using wav2vec based forced alignment.
import whisperx

device = 'cuda'
#alignment_model, metadata = whisperx.load_align_model(language_code=results["language"], device=device)
#result_aligned = whisperx.align(results["segments"], alignment_model, metadata, './audio_16k.wav', device)

/usr/local/lib/python3.10/dist-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


KeyboardInterrupt: 

You WhisperX alignment may fail. If this happens then it's most probably because Whisper just `hallucinated` i.e. whisper came up with extra/weird output at the end.

This usually happens with long audio files. If this happens, I'd suggest splitting big audio files in small files.

In [9]:
# transcribe the audio track using Whisper by OpenAI set _translate=False to transcribe it in the original language
import whisper
def transcribe_me(_video, _translate=False):
  # setting options to define if translating or transcribing
  if _translate:
    _options = dict(task="translate", beam_size=5, best_of=5)
  else:
    _options = dict(task="transcribe", beam_size=5, best_of=5)
  model = whisper.load_model("large-v2")  # for english the .en tend to perform better otherwise just use "base"
  result = model.transcribe(_video, **_options)

  return result, result["text"], result["segments"]

In [ ]:


# store the the entire result (output), the extracted text (extracted_text) and the segments (res)
file_path= './audio_16k.wav'
output, extracted_text, res  = transcribe_me(file_path)
extracted_text

In [ ]:
# Storing words <> timestamps mapping in a file.
import json

with open('./word_ts.text', 'w+') as f:
    for line in extracted_text : #result_aligned['word_segments']:
        line_temp = line.copy()
        # WhisperX don't put a space after word but just to make sure.
        line_temp['text'] = line_temp['text'].strip()
        f.write(f'{json.dumps(line_temp)}\n')

# Speaker Diarization (Nvidian NEMO)

In this section, we'll find timestamps <> Speaker Labels mapping using Nvidia NEMO.

In [ ]:
!apt install sox libsndfile1

In [ ]:
!pip install --upgrade hydra-core llvmlite omegaconf --ignore-installed

In [ ]:
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@main

In [ ]:
!pip install --upgrade Cython jiwer braceexpand webdataset librosa sentencepiece
!pip install --upgrade youtokentome pyannote-audio transformers pandas inflect editdistance

In [ ]:
!pip install -U pytorch-lightning

You may have to restart the runtime at this stage for pytorch-lightning to take effect. Otherwise you may see errors like `Couldn't find lightning.core or lightining.logging`.

In [ ]:
import os
import json

diarize_manifest = {
  'audio_filepath': f'./audio_16k.wav',
  'offset': 0,
  'duration':  None,
  'label': "infer",
  'text': "-",
  'num_speakers': None,
  'rttm_filepath': f'./diarized/pred_rttms/audio_16k.rttm',
  'uniq_id': ""
}

if not os.path.exists('./manifest.json'):
  with open('./manifest.json', 'w') as f:
    f.write(json.dumps(diarize_manifest))

In [ ]:
import os
import wget
from omegaconf import OmegaConf
MODEL_CONFIG = os.path.join('./','diar_infer_meeting.yaml')
if not os.path.exists(MODEL_CONFIG):
    config_url = "https://raw.githubusercontent.com/NVIDIA/NeMo/main/examples/speaker_tasks/diarization/conf/inference/diar_infer_meeting.yaml"
    MODEL_CONFIG = wget.download(config_url, './')

config = OmegaConf.load(MODEL_CONFIG)

In [ ]:
config.num_workers = 4
config.batch_size = 32

config.diarizer.manifest_filepath = './manifest.json'
config.diarizer.out_dir = os.path.join('./', 'diarized')
config.diarizer.speaker_embeddings.model_path = 'titanet_large'
config.diarizer.speaker_embeddings.parameters.window_length_in_sec = [1.5, 1.0, 0.5]
config.diarizer.speaker_embeddings.parameters.shift_length_in_sec = [0.75, 0.5, 0.25]
config.diarizer.speaker_embeddings.parameters.multiscale_weights = [0.33, 0.33, 0.33]
config.diarizer.speaker_embeddings.parameters.save_embeddings = False

config.diarizer.ignore_overlap = False
config.diarizer.oracle_vad = False
config.diarizer.collar = 0.25


config.diarizer.vad.model_path = 'vad_multilingual_marblenet'
config.diarizer.oracle_vad = False # ----> Not using oracle VAD

In [ ]:
from nemo.collections.asr.models.msdd_models import ClusteringDiarizer

model = ClusteringDiarizer(cfg=config)

In [ ]:
model.diarize()

In [ ]:
# Reading timestamps <> Speaker Labels mapping

speaker_ts = []
with open('./diarized/pred_rttms/audio_16k.rttm', 'r') as f:
    lines = f.readlines()
    for line in lines:
        line_list = line.split(' ')
        s = int(float(line_list[5]) * 1000)
        e = s + int(float(line_list[8]) * 1000)
        speaker_ts.append([s, e, int(line_list[11].split('_')[-1])])


# Post-processing

From previous cells, we have got `word_ts` (words <> timestamps mapping) and `speaker_ts` (speaker label <> timestamps mapping). In this cell, we are going
to merge them together to get a list of (word, speaker label, timestamps) tuples.

In [ ]:
# Reading words <> timestamps mapping, which we saved earlier
import json

word_ts = []
with open('./word_ts.text', 'r+') as f:
    for line in f:
        line_temp = json.loads(line)
        word_ts.append(line_temp)

In [ ]:
def get_word_ts_anchor(s, e, option='start'):
  if option == 'end':
    return e
  elif option == 'mid':
    return (s + e) / 2
  return s

def get_words_speaker_mapping(wrd_ts, spk_ts, word_anchor_option='start'):
    s, e, sp = spk_ts[0]
    wrd_pos, turn_idx = 0, 0
    wrd_spk_mapping = []
    for wrd_dict in wrd_ts:
        ws, we, wrd = int(wrd_dict['start'] * 1000), int(wrd_dict['end'] * 1000), wrd_dict['text']
        wrd_pos = get_word_ts_anchor(ws, we, word_anchor_option)
        while wrd_pos > float(e):
            turn_idx += 1
            turn_idx = min(turn_idx, len(spk_ts) - 1)
            s, e, sp = spk_ts[turn_idx]
        wrd_spk_mapping.append({'word': wrd, 'start_time': ws, 'end_time': we, 'speaker': sp})
    return wrd_spk_mapping

In [ ]:
wsm = get_words_speaker_mapping(word_ts, speaker_ts, 'start')

To get accurate diarization results, we rely on correct sentence-ending punctuation i.e. period(.), question mark(?) and, exclamation(!). In all the test-cases, which I have performed using this diarization technique, Whisper was able to come up with correct punctuation markings. But there are chances for whisper to get stuck in no-punctuation loop.

One way to fix this is to use a neutral initial prompt with punctuations e.g. `model.transcribe(FILE_NAME, initial_promot='How are you doing? I hope everything is OK.')`. This approach has couple of disadvantages:



*   It could affect the transcription quality e.g. for one of my test cases, using `initial_promot` gave me `founders week real quick`, whereas the ground truth was `founders we'd work with.`
*   Transcription timestamps could be out-of-sync. For one of the examples, I observed a lag of 2 seconds which could be fixed only if we knew beforehand how much that'd be.

A better approach is to use a transformer based model to restore punctuation of the transcript.



In [ ]:
!pip install deepmultilingualpunctuation

In [ ]:
from deepmultilingualpunctuation import PunctuationModel

punct_model = PunctuationModel()
words_list = list(map(lambda x: x['word'], wsm))

labled_words = punct_model.predict(words_list)

In [ ]:
# Whisper already punctuates the text in most of the case, so we'll give priority
# to its puntuation marks over PunctuationModel results.
import re

ending_puncts = '.?!'
model_puncts = '.,;:!?'

# We don't want to punctuate U.S.A. with a period. Right?
is_acronym = lambda x: re.fullmatch(r"\b(?:[a-zA-Z]\.){2,}", x)

for word_dict, labeled_tuple in zip(wsm, labled_words):
    word = word_dict['word']
    if word and labeled_tuple[1] in ending_puncts and (word[-1] not in model_puncts or is_acronym(word)):
        word += labeled_tuple[1]
        if word.endswith('..'): word = word.rstrip('.')
        word_dict['word'] = word

# Realignment using N-Gram/Masked Language Models

In previous cells, we were able to get (word, timestamps, speaker_label) tuples which are also properly punctuated. Now we are in position to build a diarized transcription. Something like following:

```
Speaker A: Hey What's up?
Speaker B: Yeah everything is good. How about you?
.....
```

Wihtout doing any pre-processing, we'll see following results more than often.

```
Speaker A: It's got to come from somewhere else. Yeah, that one's also fun because you know the lows are
Speaker B: going to suck, right? So it's actually it hits you on both sides.
.....
```

Or something like:

```
Speaker A: That was an amazon podcast. Oh
Speaker B: yeah, I really enjoyed it.
.....
```

So you will see a splitted sentence on change of speaker. This could happen due to multiple reasons e.g. Overlapped speech, background noise or faulty timestamps (either from ASR or Diarization model)

There are couple of ways to fix that. One simple way is to use an n-gram LM (KenLM) or a masked LM to calculate sentence perplexity (Log probability). To do that, we'll get last few words of Speaker A utterance and first few words of Speaker B utterences, add `\n` at different places and calculate the sentence perplexity. Optimal position of `\n` would be where probability is maximum. Something like following:

```
Speaker A: That was an amazon
Speaker B: podcast. Oh yeah, I really enjoyed it.
perplexity_score: 1

Speaker A: That was an amazon podcast.
Speaker B: Oh yeah, I really enjoyed it.
perplexity_score: 5

Speaker A: That was an amazon podcast. Oh
Speaker B: yeah, I really enjoyed it.
perplexity_score: 3

Speaker A: That was an amazon podcast. Oh yeah.
Speaker B: I really enjoyed it.
perplexity_score: 4

Speaker A: That was an amazon podcast. Oh yeah. I
Speaker B: really enjoyed it.
perplexity_score: 1
.....
```

I tried this approach on multiple examples but results were not consistent.


# Realignment using Punctuations

Another simple method (You can call it a hack) is to using puctuation markings. If a sentence is splitted between two different speakers, it can either be part of Speaker A utterance or Speaker B utterance. We cans simply take mode of speaker labels and use that speaker label for whole sentence. Something like following:

```
Speaker A: It's got to come from somewhere else. Yeah, that one's also fun because you know the lows are
Speaker B: going to suck, right? So it's actually it hits you on both sides.
```

Speaker A has more part of the sentence `Yeah, that one's also ..... right?` so this whole sentence should be part of Speaker A utterance.

This can gets interesting when one speaker is saying some sort of monologue and other is doing `Hmm`, `Yeah`, `Exactly` in background. Whisper will tend to ignore that but Diarization Model (Nemo) will pick it up. So you may get inconsistent outputs like following:

```
Speaker A: It's got to come from somewhere
Speaker B: else. Yeah,
Speaker A: that one's also fun because you know the
Speaker C: lows are
Speaker A: going to suck, right?
```

So only Speaker A was speaking. Other's `hmm`, `yeah` were ignored by Whisper but because Diarization model was able to give us timestamps for them, we saw this inconsistent output.

Fortunately, our punctuation based realignment will fix that.

There are some special cases which are hard to fix i.e. when we have equal part of both sentences splitted between two Speakers. Something like

```
Speaker A: Oh
Speaker B: Yeah.
Speaker A: Yeah
Speaker B: Exactly.
```

Unfortunately these can't be fixed and we have to accept them as part of diarization error. Our realignment code will assign them to either of the speaker.

In [ ]:
sentence_ending_punctuations = '.?!'

def get_first_word_idx_of_sentence(word_idx, word_list, speaker_list, max_words):
  is_word_sentence_end = lambda x: x >= 0 and word_list[x][-1] in sentence_ending_punctuations
  left_idx = word_idx
  while (left_idx > 0 and word_idx - left_idx < max_words and
          speaker_list[left_idx - 1] == speaker_list[left_idx] and
          not is_word_sentence_end(left_idx - 1)):
      left_idx -= 1

  return left_idx if left_idx == 0 or is_word_sentence_end(left_idx - 1) else -1

def get_last_word_idx_of_sentence(word_idx, word_list, max_words):
  is_word_sentence_end = lambda x: x >= 0 and word_list[x][-1] in sentence_ending_punctuations
  right_idx = word_idx
  while (right_idx < len(word_list) and right_idx - word_idx < max_words and
          not is_word_sentence_end(right_idx)):
      right_idx += 1

  return right_idx if right_idx == len(word_list) - 1 or is_word_sentence_end(right_idx) else -1

def get_realigned_ws_mapping_with_punctuation(word_speaker_mapping, max_words_in_sentence = 50):
  is_word_sentence_end = lambda x: x >= 0 and word_speaker_mapping[x]['word'][-1] in sentence_ending_punctuations
  wsp_len = len(word_speaker_mapping)

  words_list, speaker_list = [], []
  for k, line_dict in enumerate(word_speaker_mapping):
      word, speaker = line_dict['word'], line_dict['speaker']
      words_list.append(word)
      speaker_list.append(speaker)

  k = 0
  while k < len(word_speaker_mapping):
      line_dict = word_speaker_mapping[k]
      if k < wsp_len - 1 and speaker_list[k] != speaker_list[k + 1] and not is_word_sentence_end(k):
          left_idx = get_first_word_idx_of_sentence(k, words_list, speaker_list, max_words_in_sentence)
          right_idx = get_last_word_idx_of_sentence(k, words_list, max_words_in_sentence - k + left_idx - 1) if left_idx > -1 else -1
          if min(left_idx, right_idx) == -1:
              k += 1
              continue

          spk_labels = speaker_list[left_idx: right_idx + 1]
          mod_speaker = max(set(spk_labels), key=spk_labels.count)
          if spk_labels.count(mod_speaker) < len(spk_labels) // 2:
              k += 1
              continue

          speaker_list[left_idx: right_idx + 1] = [mod_speaker] * (right_idx - left_idx + 1)
          k = right_idx

      k += 1

  k, realigned_list = 0, []
  while k < len(word_speaker_mapping):
      line_dict = word_speaker_mapping[k].copy()
      line_dict['speaker'] = speaker_list[k]
      realigned_list.append(line_dict)
      k += 1


  return realigned_list

In [ ]:
def get_sentences_speaker_mapping(word_speaker_mapping, spk_ts):
  s, e, spk = spk_ts[0]
  prev_spk = spk

  snts = []
  snt = {'speaker': f'Speaker {spk}', 'start_time': s, 'end_time': e, 'text': ''}

  for wrd_dict in word_speaker_mapping:
      wrd, spk = wrd_dict['word'], wrd_dict['speaker']
      s, e = wrd_dict['start_time'], wrd_dict['end_time']
      if spk != prev_spk:
          snts.append(snt)
          snt = {'speaker': f'Speaker {spk}', 'start_time': s, 'end_time': e, 'text': ''}
      else:
          snt['end_time'] = e
      snt['text'] += wrd + ' '
      prev_spk = spk

  snts.append(snt)
  return snts

def get_speaker_aware_transcript(sentences_speaker_mapping):
  with open('diarization.txt', 'w') as f:
    for sentence_dict in sentences_speaker_mapping:
        sp = sentence_dict['speaker']
        text = sentence_dict['text']
        f.write(f'\n\n{sp}: {text}')

In [ ]:
wsm = get_realigned_ws_mapping_with_punctuation(wsm)
ssm = get_sentences_speaker_mapping(wsm, speaker_ts)
get_speaker_aware_transcript(ssm)